In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models, preprocessing

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!unzip gdrive/MyDrive/pawpatrol/petfinder-pawpularity-score.zip

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Archive:  gdrive/MyDrive/pawpatrol/petfinder-pawpularity-score.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission.csv   
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace test/4128bae22183829d2b5fea10effdb0c3.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test/4128bae22183829d2b5fea10effdb0c3.jpg  
replace test/43a2262d7738e3d420d453815151079e.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test/43a2262d7738e3d420d453815151079e.jpg  
replace test/4e429cead1848a298432a0acad014c9d.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test/4e429cead1848a298432a0acad014c9d.jpg  
replace test/80bc3ccafcc51b66303c2c263aa38486.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace test/80bc3ccafcc51b66303c

In [2]:
url = 'https://raw.githubusercontent.com/Veer56/Pawl_patrol/main/Code/data/train.csv'
train_data = pd.read_csv(url)
train_data['Pawpularity'] = train_data['Pawpularity'] / 100
display(train_data.head(5))

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,0.63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,0.42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,0.28
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,0.15
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,0.72


In [3]:
import os
import cv2
import numpy as np

def load_images_from_folder(folder, data = [], size=128):
    images = []
    labels = []
    names = []
    for filename in os.listdir(folder):
        name = filename.strip('.jpg')
        names.append(name)
        labels.append(data.loc[data['Id']== name]['Pawpularity'])
        img = cv2.resize(cv2.imread(os.path.join(folder,filename)), (size, size))
        if img is not None:
            images.append(img)
    return np.array(images), labels, names

image_images, image_labels, image_names = load_images_from_folder('train', data = train_data)

print(f'Training labels loaded: {image_images.shape}')

Training labels loaded: (9912, 128, 128, 3)


In [4]:
image_images = image_images / 255


In [5]:
from sklearn.model_selection import train_test_split
image_train, image_test, label_train, label_test, name_train, name_test = train_test_split(image_images, image_labels, image_names, test_size=0.33, random_state=42)

In [6]:
def train_and_evaluate(model, train_x, train_y, val_x, val_y, preprocess={}, epochs=20, augment={}):

    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

    train_gen = preprocessing.image.ImageDataGenerator(**preprocess, **augment)
    train_gen.fit(train_x) 

    val_gen = preprocessing.image.ImageDataGenerator(**preprocess)
    val_gen.fit(train_x)

    history = model.fit(train_gen.flow(train_x, train_y), epochs=epochs, 
                        validation_data=val_gen.flow(val_x, val_y))

    fig, axs = plt.subplots(1, 2, figsize=(20, 5)) 

    for i, metric in enumerate(['loss', 'accuracy']):
        axs[i].plot(history.history[metric])
        axs[i].plot(history.history['val_' + metric])
        axs[i].legend(['training', 'validation'], loc='best')

        axs[i].set_title('Model '+metric)
        axs[i].set_ylabel(metric)
        axs[i].set_xlabel('epoch')

    plt.show()

    print(f"Validation Accuracy: {model.evaluate(val_gen.flow(val_x, val_y))[1]}")

In [7]:
model = models.Sequential()
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='linear'))


train_and_evaluate(model, image_train, label_train, image_test, label_test)

Epoch 1/20
208/208 [==============================] - 46s 168ms/step - loss: 42.2932 - accuracy: 0.0023 - val_loss: 0.0456 - val_accuracy: 0.0000e+00
Epoch 2/20
208/208 [==============================] - 33s 159ms/step - loss: 0.0445 - accuracy: 4.5174e-04 - val_loss: 0.0435 - val_accuracy: 0.0000e+00
Epoch 3/20
208/208 [==============================] - 33s 159ms/step - loss: 0.0455 - accuracy: 0.0014 - val_loss: 0.0441 - val_accuracy: 0.0000e+00
Epoch 4/20
208/208 [==============================] - 33s 159ms/step - loss: 0.0426 - accuracy: 4.5174e-04 - val_loss: 0.0465 - val_accuracy: 0.0000e+00
Epoch 5/20
101/208 [=============>................] - ETA: 15s - loss: 0.0396 - accuracy: 0.0016

KeyboardInterrupt: ignored